In [1]:
import os
import cdsw
from src.cml_extensions.dask_cluster.dask_cluster import DaskCluster

#os.environ[""]

cluster = DaskCluster(num_workers=10, worker_cpu=16, worker_memory=36, scheduler_cpu=16, scheduler_memory=36)
cluster.init()


--------------------
Dask cluster started
--------------------

The Dask dashboard is running at
https://dwjiseoem170fd7x.ml-6804005e-1f7.paul-aug.a465-9q4k.cloudera.site/status

To connect to this Dask cluster from this CML Session,
use the following Python code:
  from dask.distributed import Client
  client = Client('tcp://100.100.220.247:8786')



In [4]:
from multiprocessing import cpu_count

In [5]:
cpu_count()

96

In [6]:
from dask.distributed import Client
client = Client('tcp://100.100.220.247:8786')

client

<Client: 'tcp://100.100.220.247:8786' processes=10 threads=960, memory=334.65 GiB>

## Read Sample Dataset

In [7]:
import dask
df = dask.datasets.timeseries()
df

/home/cdsw/.local/lib/python3.11/site-packages/dask/dataframe/__init__.py:42: FutureWarning: 
Dask dataframe query planning is disabled because dask-expr is not installed.

You can install it with `pip install dask[dataframe]` or `conda install dask`.
This will raise in a future version.

  warnings.warn(msg, FutureWarning)


,name,id,x,y
npartitions=30,,,,
2000-01-01,string,int64,float64,float64
2000-01-02,...,...,...,...
...,...,...,...,...
2000-01-30,...,...,...,...
2000-01-31,...,...,...,...


## Read Parquet from S3

In [31]:
import os
import dask.dataframe as dd

df = dd.read_parquet("s3://paul-aug26-buk-a3c2b50a/data/pdefusco/modin/100cols")
df.head()

,col1,col2,col3,col4,col5,col6,col7,col8,col9,col10,...,col90,col91,col92,col93,col94,col95,col96,col97,col98,col99
0,A,2155749.0,2162552.0,8879422.0,3592760.0,6260197.0,4380193.0,649749.0,8567364.0,7842953.0,...,4659230.0,554457.0,5490286.0,6592815.0,2679440.0,1867962.0,3886603.0,2371716.0,3779632.0,32281.0
1,B,1461406.0,6489945.0,1028377.0,252657.0,7100934.0,1732775.0,9956641.0,3115993.0,4463726.0,...,2031911.0,749221.0,5082530.0,6681090.0,8306104.0,3721765.0,6999662.0,707691.0,2327278.0,1639487.0
2,C,1327473.0,8855601.0,9027891.0,6230956.0,3983144.0,1449433.0,3296800.0,4257279.0,8776295.0,...,5439053.0,5520726.0,2956889.0,1553031.0,2366965.0,4412799.0,9584188.0,3858757.0,344177.0,4615917.0
3,D,4750508.0,7597960.0,6513623.0,2545998.0,252819.0,8349154.0,8894517.0,6153052.0,2255401.0,...,3519633.0,5806606.0,7916276.0,8496809.0,9879546.0,8856266.0,8577436.0,824027.0,2011149.0,6533642.0
4,E,7268317.0,6482051.0,8168993.0,7054196.0,3747121.0,6873729.0,2527835.0,1162200.0,2718198.0,...,964083.0,66924.0,7246620.0,1870502.0,6532386.0,6427307.0,4309461.0,8788504.0,2979797.0,7008068.0


## Dask Distributed Dataframe Operations

In [32]:
df.info()

<class 'dask.dataframe.core.DataFrame'>
Columns: 99 entries, col1 to col99
dtypes: float32(98), string(1)

In [33]:
%time df.groupby('col1').col2.mean().compute()

CPU times: user 16.3 ms, sys: 379 µs, total: 16.7 ms
Wall time: 298 ms


col1
A    5.049980e+06
B    5.020121e+06
C    4.993238e+06
D    5.009756e+06
E    5.070729e+06
F    5.103563e+06
G    4.945406e+06
Name: col2, dtype: float64

## Distributed Machine Learning

#### KMeans

In [34]:
import dask_ml.cluster
import matplotlib.pyplot as plt

In [35]:
X = df.drop("col1", axis=1)

In [36]:
km = dask_ml.cluster.KMeans(n_clusters=3, init_max_iter=2, oversampling_factor=10)
km.fit(X)

/home/cdsw/.local/lib/python3.11/site-packages/dask/base.py:1541: UserWarning: Running on a single-machine scheduler when a distributed client is active might lead to unexpected results.
  warnings.warn(


KMeans(init_max_iter=2, n_clusters=3, oversampling_factor=10)

#### Incremental Classification

In [90]:
from dask_ml.model_selection import train_test_split
y = df.col1
X_train, X_test, y_train, y_test = train_test_split(X, y)
X_train

/home/cdsw/.local/lib/python3.11/site-packages/dask_ml/model_selection/_split.py:464: FutureWarning: The default value for 'shuffle' must be specified when splitting DataFrames. In the future DataFrames will automatically be shuffled within blocks prior to splitting. Specify 'shuffle=True' to adopt the future behavior now, or 'shuffle=False' to retain the previous behavior.
  warnings.warn(


,col2,col3,col4,col5,col6,col7,col8,col9,col10,col11,col12,col13,col14,col15,col16,col17,col18,col19,col20,col21,col22,col23,col24,col25,col26,col27,col28,col29,col30,col31,col32,col33,col34,col35,col36,col37,col38,col39,col40,col41,col42,col43,col44,col45,col46,col47,col48,col49,col50,col51,col52,col53,col54,col55,col56,col57,col58,col59,col60,col61,col62,col63,col64,col65,col66,col67,col68,col69,col70,col71,col72,col73,col74,col75,col76,col77,col78,col79,col80,col81,col82,col83,col84,col85,col86,col87,col88,col89,col90,col91,col92,col93,col94,col95,col96,col97,col98,col99
npartitions=10,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32
,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...


In [96]:
import dask.array as da
from dask_ml import preprocessing

le = preprocessing.LabelEncoder()
le.fit(y_train)
y_train_num = le.transform(y_train)

/home/cdsw/.local/lib/python3.11/site-packages/dask/dataframe/core.py:3769: UserWarning: Dask currently has limited support for converting pandas extension dtypes to arrays. Converting string to object dtype.
  warnings.warn(
/home/cdsw/.local/lib/python3.11/site-packages/dask/dataframe/core.py:3769: UserWarning: Dask currently has limited support for converting pandas extension dtypes to arrays. Converting string to object dtype.
  warnings.warn(


In [99]:
classes = da.unique(y_train_num).compute()
classes

array([0, 1, 2, 3, 4, 5, 6])

In [ ]:
from sklearn.linear_model import SGDClassifier
from dask_ml.wrappers import Incremental

est = SGDClassifier(loss='log', penalty='l2', tol=1e-3)
inc = Incremental(est, scoring='accuracy')